In [1]:
import pandas as pd
from Analiza_pianek import analiza_pianek as ap, owaty as ow

analiza = ap.analiza
komplety_pianek = ap.komplety_pianek
pda = ap.pda
zam_pianki = ap.zam_pianki
owaty = ow.owaty

{'path_dane_pianki': 'Z:/450. PLANISTA - ZAOPATRZENIE/DANE_PIANKI_2409.xlsx', 'owaty': 'Z:/450. PLANISTA - ZAOPATRZENIE/111 KROJOWNIA SUROWKI PIANKI OWATY.xlsx', 'zam_pianki_link': 'Z:/450. PLANISTA - ZAOPATRZENIE/ZAM_PIANKI.xlsx'}


In [2]:
# @title BRYŁY DO ZAMÓWIENIA FUNKCJE
def oblicz_owaty_do_zamowienia(*z_bdz):

  df_zam = pd.concat(z_bdz)
  df_zam = df_zam.merge(owaty, how="left", on="OPIS")

  for c in owaty.columns[1:]:
    df_zam[c] = df_zam[c] * df_zam.DO_ZAMOWIENIA * 1.1

  return df_zam#[owaty.columns[1:]]


def zapotrzebowanie_na_owaty(zam_owaty, wyjatki:list):
  _wyjatki = pd.Series({"O3":0, "O2":0, "O1":0, "L1":0, "W3":0})
  for w in wyjatki:
    _wyjatki += w

  zap = zam_owaty[["O3", "O2", "O1", "L1", "W3"]].sum()+_wyjatki

  # zap *= 1.1

  print(f"O1 zielona: {(zap['O1']/50).round(0):.0f} rolek")
  print(f"O2 niebieska: {(zap['O2']/40).round(0):.0f} rolek")
  print(f"O3 czerwona: {(zap['O3']/40).round(0):.0f} rolek")
  # return zap



def pobierz_zamowienie_z_ZAM_PIANKI(tydzien, _cls):
  _zam = zam_pianki[(zam_pianki.TYDZIEN == tydzien) & (zam_pianki.OPIS.str.contains(_cls.MODEL))].merge(komplety_pianek[["KOD", "BRYLA_GEN"]])[["OPIS", "BRYLA_GEN", "ILE_ZAMOWIONE"]]
  return _cls({i[1].BRYLA_GEN: i[1].ILE_ZAMOWIONE for i in _zam[["BRYLA_GEN", "ILE_ZAMOWIONE"]].iterrows()})


In [ ]:
zapotrzebowanie_na_owaty(oblicz_owaty_do_zamowienia(zsto, zhor),[])

pd.concat([zsto, zhor]).merge(
    oblicz_owaty_do_zamowienia(zsto, zhor)[["KOD", "O1", "O2", "O3", "L1","W3"]].fillna(0),
    how="left",
    on="KOD").rename(columns={"O1":"ZIELONA", "O2":"NIEBIESKA", "O3":"CZERWONA", "L1":"ŻÓŁTA"})#.to_excel("PIANPOL_2408.xlsx")

In [ ]:
from modele_db import *
pozycje = list()


def Dodaj_pozycje_do_ZAM_PIANKI(tydzien, zancznik_dostawcy, nr_kompletacji, modele:pd.DataFrame, klasa, zam1=None, zam2=None, nr_partii=None, DODAJ_DO_BAZY = False):

  model = klasa.MODEL
  galanteria = klasa.galanteria
  siedziska_HR = klasa.siedziska_HR
  leniwa = klasa.leniwa

  for r in modele.iterrows():
    # tydzien, model, kod, opis, ile_zam, znacznik_dostawcy, galanteria, siedziska_HR, leniwa, nr_kompletacji=None, zam1=None, zam2=None, nr_partii=None
    pozycje.append([tydzien, model, r[1].KOD, r[1].OPIS, r[1].DO_ZAMOWIENIA, zancznik_dostawcy, galanteria, siedziska_HR, leniwa, nr_kompletacji, zam1, zam2, "nr_partii: " + nr_partii])

  if DODAJ_DO_BAZY:

    for r in modele.iterrows():
      session.add(ZAM_PIANKI(tydzien, model, r[1].KOD, r[1].OPIS, r[1].DO_ZAMOWIENIA, zancznik_dostawcy, galanteria, siedziska_HR, leniwa, nr_kompletacji, zam1, zam2, "nr_partii: " + nr_partii))

    session.commit()


#dos 24/0242 sto 2_24, hor 3_24, nr_sam VITA 7_24,  nr partii 08/01

# Dodaj_pozycje_do_ZAM_PIANKI(2407, "P", "2_24", zoxy, oxy, "24/0216", nr_partii="07/01,")#, DODAJ_DO_BAZY=True)

pd.DataFrame(pozycje)